<a href="https://colab.research.google.com/github/ml-unison/ml-unison.github.io/blob/main/ejemplos/intro-pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Introducción muy breve a `pandas`


<p> Julio Waissman Vilanova </p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/intro-pandas.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

In [ ]:
import pandas as pd

## Creando un dataframe

Pandas es un marco de desarrollo para la manipulación de datos, creado originalmente para ofrecer en python las facilidades de manejo de tablas de datos que tiene en forma nativa el lenguaje `R`. 

Pandas se basa en dos clases: `Serie` y `DataFrame`, ambas heredadas de la clase `ndarray`de numpy. Un objeto de la clase `Serie`(que llamaremos serie) es un arreglo de datos de un solo tipo, los cuales se encuentran indexados. Un objeto tipo `DataFrame`es una colección de series, en la que se comparte el índice (o renglón), pero cada serie (o columna) tiene su propio tipo. En la figura se muestra como es un DataFrame

![](https://pandas.pydata.org/docs/_images/01_table_dataframe.svg)

Vamos a crear un `Dataframe`:

In [ ]:
df = pd.DataFrame(
    {
        "Name": [
            "Braund, Mr. Owen Harris",
            "Allen, Mr. William Henry",
            "Bonnell, Miss. Elizabeth",
        ],
        "Age": [22, 35, 58],
        "Sex": ["male", "male", "female"]
    }
)

Y vamos a ver los primeros 2 elementos y los últimos 3

In [ ]:
display(df.head(2))

In [ ]:
display(df.tail(2))

Ahora vamos a investigar la información de la tabla y cada una de las series que la componen:

In [ ]:
df['Name']

In [ ]:
df['Age']

In [ ]:
df.index

In [ ]:
df.index = ['tata', 'tete', 'toto']
df

In [ ]:
df.dtypes

In [ ]:
df.describe(include='all')

In [ ]:
df.describe(include=object)

## Leyendo un dataframe

En pandas, es posible leer y escribir los dataframes en diferentes formatos. Para esto en pandas hay una serie de funciones `read_*` y `to_*` dependiendo el formato en que estén los datos o en que queramos leerlos. 

Todos los formatos tienen sus cositas y es prudente leer con calma la documentación (inclusive si se trata de abrir un archivo `csv`y es muy grande o está guardado en una codificación extraña o antigua).

En la figura vemos algunos de los formatos de lectura y escritura existentes.

![](https://pandas.pydata.org/docs/_images/02_io_readwrite.svg)

En pandas, por ejemplo es posible abrir un archivo local, remoto o inclusive comprimido, sin necesidad de pasar por otros pasos. Esto hace que sea fácil utilizar datos en colab (datos públicos) leyendolos de un repositorio de github.

Vamos pues a leer los sobadisimos e interesantes datos del titanic:

In [ ]:
%pwd

In [ ]:
csv_titanic_url = "https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv"
local_titanic_filename = "datos/titanic.csv"

df_titanic = pd.read_csv(csv_titanic_url, engine='python')
df_titanic.head(5)

Y por buena costumbre vamos a ver de que se tratan estos datos

In [ ]:
df_titanic.info()

In [ ]:
df_titanic.describe()

In [ ]:
df_titanic.describe(include=object)

Y ahora vamos a guardar los datos en un archivo excel para mandarselos a alguien (recuerda que colab mantiene los archivo en el entorno virtual, pero si no los guardas despues, se pierden).

In [ ]:
df_titanic.to_excel("datos/titanic.xlsx", sheet_name="passengers", index=False)

## Seleccionando partes de un dataframe

### Seleccionando columnas

Seleccionar columnas es muy fácil, solo hay que tener presente que si se selecciona una sola columna, lo que se obtiene es una serie, mientras que si se selecciona un subconjunto de columnas, lo que se obtiene es otro dataframe. Quedarse con un subconjunto de columnas se conoce tambien como seleccionar.

![](https://pandas.pydata.org/docs/_images/03_subset_columns.svg)

Vamos a ver que pasa:

In [ ]:
edad = df_titanic['Age']
edad_bis = df_titanic.Age   # Es lo mismo que el anterior

df_edad = df_titanic[['Age']] # Un subconjunto de columnas con una sola columna

df_ejemplo = df_titanic[['Age', 'Sex']]

In [ ]:
edad

In [ ]:
type(edad)

In [ ]:
edad_bis

In [ ]:
type(edad_bis)

In [ ]:
df_edad

In [ ]:
type(df_edad)

In [ ]:
df_ejemplo

### Seleccionando renglones

Los renglones tienen mas detallitos a tomar en cuenta que las columnas. 
Este proceso se conoce en general como filtrado, y lo que se busca es seleccionar solo los 
renglones que cumplan ciertos criterios. Veamos.

Vamos empezando por buscar un dataframe de la información de los pasageros con 35 años o mas:

In [ ]:
df_35mas = df_titanic[df_titanic['Age'] > 35]   #equivalente df_titanic.Age[df_titanic.Age > 35]

df_35mas.describe()

Ahora vamos a buscar los pasajeros que se viajaron en 1ra o 2da clase:

In [ ]:
df_12 = df_titanic[df_titanic.Pclass.isin([1, 2])]

df_12.describe()

Se pueden usar combinadores lógicos entre diferentes columnas, pero las condiciones debe estar 
clara con el uso de paréntesis y se deben utilizar `|` para la disyunción y `&` para la conjunción.

Vamos a ver los pasajeros de más de 35 años y que viajen en 1ra o 2da:

In [ ]:
df_titanic[(df_titanic.Age > 35) & (df_titanic.Pclass.isin([1, 2]))].describe()

Por últomo vamos a ver los pasajeron que no registraron su edad:

In [ ]:
df_sin_edad = df_titanic[df_titanic.Age.isna()]

df_sin_edad

### Seleccionando renglones y columnas

![](https://pandas.pydata.org/docs/_images/03_subset_columns_rows.svg)

Este paso es un poco extraño, ya que no se pueden seleccionar renglones y columnas directamente, sino que hay que usar los métodos `.loc`y `.iloc`. Vamos a ejemplificarlos.

Supongamos que queremos los nombres de todos los mayores de 35 años, como serie y como dataframe. La manera de seleccionar y filtrar es la siguiente:

In [ ]:
como_serie = df_titanic.loc[df_titanic.Age > 35, 'Name']  # Como serie de tiempo

como_df = df_titanic.loc[df_titanic.Age > 35, ['Name']]  # Como dataframe

In [ ]:
como_serie

In [ ]:
como_df

Si quieres no solo el nombre, pero tambien el género se puede obtener como:

In [ ]:
df_nueva = df_titanic.loc[df_titanic.Age > 35, ['Name', 'Sex']]
df_nueva

Este tipo de indexación tambien puede servir para modificar valores, por ejemplo:

In [ ]:
df_nueva.loc[df_nueva.Sex == 'female', 'Sex'] = 'mujer'
df_nueva

## Extrayendo estadísticas

### Por columna

Para encontrar estadísticas podemos seleccionar una columna y aplicarle cualquier oeración de agregación incluida en las operaciones en series. Por ejemplo, para encontrar la edad promedio de los pasajeros del titanic:

In [ ]:
df_titanic['Age'].mean()

Las operaciones de agregación tambien se pueden aplicar en dataframes, aplicandose en cada serie en forma independiente. Por ejemplo:

In [ ]:
df_titanic[['Age', 'Fare']].median()

Y si se quiere aplicar una serie de agregaciones diferentes a varias variables, lo mejor es usar el método `.agg`como se muestra a continuación: 

In [ ]:
df_titanic.agg(
    {
        "Age": ["min", "max", "median", "mad"],
        "Fare": ["min", "max", "mean", "std"],
    }
)

### Regrupando por variables

Para esto se usa el método `.group` que si bien de inicio parece bastante obvio, luego se le ven algunos detallitos.

Por ejemplo, supongamos que quiero saber la edad pronedio de los pasajeros por género. Hay dos maneras de hacerlo. La primera es seleccionar `Age` y `Sex`, regrupar por `Sex`y sacarle la media a Àge`.

![](https://pandas.pydata.org/docs/_images/06_groupby.svg)

La otra es regrupar por `Sex`, luego seleccionar `Age`, y aplicarle la media.

![](https://pandas.pydata.org/docs/_images/06_groupby_select_detail.svg)

Vamos a ver que sale en cada caso:

In [ ]:
df_titanic[['Age', 'Sex']].groupby('Sex').mean()

In [ ]:
df_titanic.groupby('Sex')[['Age']].mean()

Se pueden hacer regrupaciones en mútiples niveles. Por ejemplo si queremos saber la edad promedio, por género y por clase en la que viajaban, se puede regrupar en dos variables:

In [ ]:
df_titanic.groupby(["Sex", "Pclass"])[["Age"]].mean()

## Datos numéricos

Para ejemplificar el uso de columnas numéricas, horas y fechas vamos a utilizar un conjunto de datos que puso generosamente a nuestra disposición [Hector Alberto Gutierrez Ibarra](hector.gutierrez@cenace.gob.mx) de la gerencia noroeste del *Centro Nacional de Control de Energía (CENACE)*.

Vamos primero pegandole un ojo a los datos:

In [ ]:
path_datos = "https://github.com/juliowaissman/curso-python-cd/raw/main/datos/caso_zc_hmo.csv.zip"
df = pd.read_csv(path_datos)
print(df.info())
df

Lo primero que tenemos que hacer es convertir la variable `Date`en un formato de fechas. En este caso es muy fácil porque la tabla está muy bien formateada, pero suele no ser tan sencillo.

df['Date'] = pd.to_datetime(
    df.Date, 
    format="%d/%m/%Y %H:%M"
)
print(df.info())
df

¿Cuantos días tenemos en esta base? ¿Cual es el primer día y el último?

In [ ]:
print(f"Inicia el {df.Date.min()} y termina el {df.Date.max()}")
print(f"Con una duración de {df.Date.max() - df.Date.min()}")

Hay que tener cuidado porque hay dos tipos fundamentales de formatos de tiempo, y suelen no mesclarse bien en algunas operaciones.

In [ ]:
type(df.Date.min()), type(df.Date.max() - df.Date.min())

Para facilitar el uso de pandas, vamos a pasar la fecha como el indice del dataframe

In [ ]:
df.set_index('Date', inplace=True)
df

## Gráficas rápidas y furiosas desde pandas

Pandas trae incluidas facilidades para la graficación con el fin de hacer análisis rápidos de nuestras variables, así que vamos aprovechando y haciendo una inspección visual

In [ ]:
ax = df.Demand.plot()

In [ ]:
ax = df.plot(figsize=(12, 20), subplots=True)

In [ ]:
ax = df.Temperature.plot.box()

In [ ]:
ax = df.plot.scatter(
    x='Humidity',
    y='Temperature',
    c='Demand',
    s =40,
    figsize=(12,8)
)

## Generando nuevas variables a partir de las variables conocidas

Generar nuevas variables es relativamente simple, pero hay algunas cosas que son diferentes con numpy y por las cuales hay que tener cuidado.

df['farenheit'] = (df.Temperature * 9/5) + 32

df['refri'] = 0
df['refri'] = df.refri.where(df.Temperature < 30, 1)

df['DiaSemana'] = df.index.weekday

df.rename(
    columns={
        'Demand': 'Demanda',
        'Temperature': 'Temperatura',
        'PrecipIntensity': 'Precipitación',
        'Humidity': 'Humedad',
        'WinSpeed': 'VelocidadViento',
    },
    inplace=True
)
df

## Analizando con regrupamientos

En particular, es interesante poder hacer algun análisis exploratorio utilizando las facilidades que da el uso del manejo de la información temporal que ofrece pandas. En espacial cuando se combina con `groupby` y diferentes funciones de agregación.

Vamos a empezar por ver la demanda promedio por día de la semana:

In [ ]:
ax = df.groupby(df.index.month)[['Demanda']].boxplot(subplots=False, rot=90)

In [ ]:
df.boxplot(column=['Demanda'], by=df.index.month)

In [ ]:
df_semana = df.groupby('DiaSemana').agg(
    {
        'Demanda': ['min', 'max', 'mean', 'median', 'std'],
        'Precipitación': ['min', 'max']
    }
)
df_semana

Ahora vamos a ver que pasa en forma mensual

In [ ]:
ax = (
    df[['Demanda']]
    .groupby(df.index.month)
    .boxplot(subplots=False, rot=90, figsize=(12, 7))
)

In [ ]:
ax = (
    df[['Temperatura']]
    .groupby(df.index.month)
    .boxplot(
        subplots=False, 
        rot=90, 
        figsize=(12, 7)
    )
)

O inclusive por hora del día

In [ ]:
ax = (
    df[['Demanda']]
    .groupby(df.index.hour)
    .boxplot(subplots=False, rot=90, figsize=(12, 7)
)

In [ ]:
df_hora = (
    df[['Demanda', 'Temperatura']]
    .groupby([df.index.hour, df.index.month_name()])
    .mean()
    .unstack()
)
df_hora

In [ ]:
ax = df_hora.Demanda.plot(
    style='o', 
    figsize=(15,7),
    title='Demanda por hora y por mes',
    xlabel='Hora',
    ylabel='Demanda'
)

In [ ]:
ax = df_hora.Temperatura.plot(
    style='o', 
    figsize=(15,7))

## Generando nuevas variables a partir de las variables conocidas

Generar nuevas variables es relativamente simple, pero hay algunas cosas que son diferentes con numpy y por las cuales hay que tener cuidado.

### Vamos a practicar

Para esta practica vamos a usar un conjunto de datos de la revista *wine magazine*, 
donde revisan una cantidad bastante sorprendente de vinos. 

Una descripción de la base de datos la encuentras [aquí](https://www.kaggle.com/zynicide/wine-reviews). 
Para no tener que descargar los datos a mano, se anexa la dirección `url` de donde se pueden descargar.

Es importante notr que la primer columna del archivo `csv` es el índice (usar `index_col=0` cuando se descargue el archivo con `pd.read_csv`). 

Una vez descargado, usar pandas para las siguientes tareas:

1. ¿Cuantas variables tiene el dataframe? ¿Qué variables tienen valores perdidos? ¿Qué variables son numéricas? ¿Qué variables son cualitativas?
2. Hacer un dataframe con únicamente vinos europeos.
3. ¿Cuál es el menor, el mayor y el precio promedio de la botella por país? ¿De que país es la botella de menor precio?
4. ¿Cuantos vinos hay con *aroma a fresa* entre otras consideraciones snobs que vienen en la descripción?
5. ¿Cuantas designaciones diferentes hay? ¿Cuál es la más repetida? ¿Cuantas veces se repite?
6. Hacer un dataframe con la variedad, el país y el precio para vinos con un costo menor a los $20 dolares.
7. ¿Cuantos vinos diferentes de la variedad *Pinot Noir* hay por cada país?


In [ ]:
winmag_url = 'https://gist.githubusercontent.com/clairehq/79acab35be50eaf1c383948ed3fd1129/raw/407a02139ae1e134992b90b4b2b8c329b3d73a6a/winemag-data-130k-v2.csv'

In [ ]:
# ¿Cuantas variables tiene el dataframe? 
# ¿Qué variables tienen valores perdidos? 
# ¿Qué variables son numéricas? 
# ¿Qué variables son cualitativas?


In [ ]:
# Hacer un dataframe con únicamente vinos europeos.


In [ ]:
# ¿Cuál es el menor, el mayor y el precio promedio de la botella por país? 
# ¿De que país es la botella de menor precio?


In [ ]:
# ¿Cuantos vinos hay con *aroma a fresa* entre otras consideraciones snobs que vienen en la descripción?


In [ ]:
# ¿Cuantas designaciones diferentes hay? ¿Cuál es la más repetida? ¿Cuantas veces se repite?


In [ ]:
# Hacer un dataframe con la variedad, el país y el precio para vinos con un costo menor a los $20 dolares


In [ ]:
# ¿Cuantos vinos diferentes de la variedad *Pinot Noir* hay por cada país?
